In [1]:
!pip install scikit-surprise


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 5.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3162742 sha256=8c75f17b46c1f5a10974cfacfa60bfa90247e666520d1d146216a5af85295ca4
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [2]:

import pandas as pd

from surprise import Dataset, SVD, SVDpp, NMF, accuracy
from surprise.model_selection import cross_validate, GridSearchCV

from collections import defaultdict
import statistics
from surprise.model_selection import train_test_split

In [3]:
data = Dataset.load_builtin("ml-100k")

Dataset ml-100k could not be found. Do you want to download it? [Y/n] Y
Trying to download dataset from https://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /root/.surprise_data/ml-100k


In [7]:
raw_ratings = data.raw_ratings
raw_ratings = [(user_id, item_id, rating) for user_id, item_id, rating, _ in raw_ratings]

columns = ['user_id', 'item_id', 'rating']
df = pd.DataFrame(raw_ratings, columns=columns)

print(df.head())

  user_id item_id  rating
0     196     242     3.0
1     186     302     3.0
2      22     377     1.0
3     244      51     2.0
4     166     346     1.0


In [8]:
mean_rating = df['rating'].mean()
std_rating = df['rating'].std()

print("Mean Rating:", mean_rating)
print("Standard Deviation of Ratings:", std_rating)

Mean Rating: 3.52986
Standard Deviation of Ratings: 1.125673599144316


In [13]:
trainset, testset = train_test_split(data, test_size=0.2)

**SVD**

In [9]:
param_grid_svd = {
    'n_epochs': [10, 15, 20],
    'lr_all': [0.002, 0.005],
    'reg_all': [0.04, 0.06],
    'n_factors': [1, 20, 50],
    'init_mean': [mean_rating],
    'init_std_dev': [0.1, 1]
}

In [10]:
grid_search_svd = GridSearchCV(SVD, param_grid_svd, measures=['rmse', 'mae'], cv=5, n_jobs=-1, joblib_verbose=5)

grid_search_svd.fit(data)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:    4.0s
[Parallel(n_jobs=-1)]: Done  68 tasks      | elapsed:   18.3s
[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed:   44.0s
[Parallel(n_jobs=-1)]: Done 284 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 360 out of 360 | elapsed:  1.9min finished


In [11]:
print(grid_search_svd.best_params)
print(grid_search_svd.best_score['rmse'])
print(grid_search_svd.best_score['mae'])
best_params_svd = grid_search_svd.best_params['rmse']
print(best_params_svd)

{'rmse': {'n_epochs': 20, 'lr_all': 0.005, 'reg_all': 0.06, 'n_factors': 1, 'init_mean': 3.52986, 'init_std_dev': 0.1}, 'mae': {'n_epochs': 20, 'lr_all': 0.005, 'reg_all': 0.06, 'n_factors': 1, 'init_mean': 3.52986, 'init_std_dev': 0.1}}
0.9513054147201618
0.7492765859945156
{'n_epochs': 20, 'lr_all': 0.005, 'reg_all': 0.06, 'n_factors': 1, 'init_mean': 3.52986, 'init_std_dev': 0.1}


In [14]:

algo_svd = SVD(n_epochs=20, n_factors=1, lr_all=0.005, reg_all=0.06, init_mean=3.52986, init_std_dev=0.1)

algo_svd.fit(trainset)

predictions = algo_svd.test(testset)

rmse = accuracy.rmse(predictions)
mae = accuracy.mae(predictions)

RMSE: 0.9488
MAE:  0.7462


**SVD++**

In [15]:
param_grid_svd_plus = {
    'n_epochs': [10, 15, 20],
    'lr_all': [0.002, 0.005],
    'reg_all': [0.04, 0.06],
    'n_factors': [1, 20, 50],
    'init_mean': [mean_rating],
    'init_std_dev': [0.1]
}

In [16]:
grid_search_svd_plus = GridSearchCV(SVDpp, param_grid_svd_plus, measures=['rmse', 'mae'], cv=3, n_jobs=-1, joblib_verbose=5)

grid_search_svd_plus.fit(data)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done  68 tasks      | elapsed: 12.6min
[Parallel(n_jobs=-1)]: Done 108 out of 108 | elapsed: 24.3min finished


In [17]:
print(grid_search_svd_plus.best_params)
print(grid_search_svd_plus.best_score['rmse'])
print(grid_search_svd_plus.best_score['mae'])
best_params_svd_plus = grid_search_svd_plus.best_params['rmse']
print(best_params_svd_plus)

{'rmse': {'n_epochs': 20, 'lr_all': 0.002, 'reg_all': 0.04, 'n_factors': 1, 'init_mean': 3.52986, 'init_std_dev': 0.1}, 'mae': {'n_epochs': 20, 'lr_all': 0.002, 'reg_all': 0.04, 'n_factors': 1, 'init_mean': 3.52986, 'init_std_dev': 0.1}}
1.7660545742972467
1.4379008379057285
{'n_epochs': 20, 'lr_all': 0.002, 'reg_all': 0.04, 'n_factors': 1, 'init_mean': 3.52986, 'init_std_dev': 0.1}


In [18]:
algo_svd_plus = SVDpp(n_epochs=20, n_factors=1, lr_all=0.002, reg_all=0.04, init_mean=3.52986, init_std_dev=0.1)

algo_svd_plus.fit(trainset)

predictions = algo_svd_plus.test(testset)

rmse = accuracy.rmse(predictions)
mae = accuracy.mae(predictions)

RMSE: 1.9725
MAE:  1.6367


**NMF**

In [19]:
param_grid_nmf = {
    'n_epochs': [10, 15, 20],
    'n_factors': [1, 20, 50],
    'reg_pu': [0.02, 0.04, 0.06],
    'reg_qi': [0.02, 0.04, 0.06]
}

In [20]:
grid_search_nmf = GridSearchCV(NMF, param_grid_nmf, measures=['rmse', 'mae'], cv=3, n_jobs=-1, joblib_verbose=5)

grid_search_nmf.fit(data)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:    5.3s
[Parallel(n_jobs=-1)]: Done  68 tasks      | elapsed:   24.2s
[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 243 out of 243 | elapsed:  1.8min finished


In [21]:
print(grid_search_nmf.best_params)
print(grid_search_nmf.best_score['rmse'])
print(grid_search_nmf.best_score['mae'])
best_params_nmf = grid_search_nmf.best_params['rmse']
print(best_params_nmf)

{'rmse': {'n_epochs': 20, 'n_factors': 20, 'reg_pu': 0.06, 'reg_qi': 0.06}, 'mae': {'n_epochs': 20, 'n_factors': 20, 'reg_pu': 0.06, 'reg_qi': 0.06}}
1.1114242644968486
0.8463821365952997
{'n_epochs': 20, 'n_factors': 20, 'reg_pu': 0.06, 'reg_qi': 0.06}


In [22]:
algo_nmf = NMF(n_epochs=20, n_factors=20, reg_pu=0.06, reg_qi=0.06)

algo_nmf.fit(trainset)

predictions = algo_nmf.test(testset)

rmse = accuracy.rmse(predictions)
mae = accuracy.mae(predictions)

RMSE: 1.0972
MAE:  0.8347


In [23]:

algos = [SVD(**best_params_svd), SVDpp(**best_params_svd_plus), NMF(**best_params_nmf)]
for algo in algos:
    results = cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)
    print(f"Algorithm: {algo.__class__.__name__}")
    print(f"RMSE: {sum(results['test_rmse']) / len(results['test_rmse'])}")
    print(f"MAE: {sum(results['test_mae']) / len(results['test_mae'])}")
    print("---")


Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9504  0.9464  0.9592  0.9526  0.9518  0.9521  0.0041  
MAE (testset)     0.7471  0.7455  0.7548  0.7518  0.7505  0.7499  0.0033  
Fit time          0.40    0.43    0.26    0.52    0.70    0.46    0.15    
Test time         0.31    0.08    0.18    0.14    0.08    0.16    0.09    
Algorithm: SVD
RMSE: 0.9520865532993291
MAE: 0.7499409707752936
---
Evaluating RMSE, MAE of algorithm SVDpp on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.8461  1.9495  1.9508  1.9838  1.9541  1.9369  0.0471  
MAE (testset)     1.4647  1.6105  1.6131  1.6439  1.6126  1.5890  0.0633  
Fit time          4.46    4.88    4.95    4.49    4.50    4.66    0.22    
Test time         3.64    3.69    3.69    4.04    3.50    3.71    0.18    
Algorithm: SVDpp
RMSE: 1.936853769552793
MAE: 1.588959054164065
---
Eval

Висновок
SVD:
Найкращі параметри: {'n_epochs': 20, 'lr_all': 0.005, 'reg_all': 0.06, 'n_factors': 1, 'init_mean': 3.52986, 'init_std_dev': 0.1}
Найкраща RMSE: 0.9517
Найкраща MAE: 0.7498
Середні часи навчання (Fit time) та часи тестування (Test time) на кожному спліті заняли відповідно приблизно 0.89 та 0.25 секунди.
SVD++:
Найкращі параметри: {'n_epochs': 20, 'lr_all': 0.002, 'reg_all': 0.04, 'n_factors': 1, 'init_mean': 3.52986, 'init_std_dev': 0.1}
Найкраща RMSE: 1.7480
Найкраща MAE: 1.4018
Середні часи навчання (Fit time) та часи тестування (Test time) на кожному спліті заняли відповідно приблизно 8.05 та 5.45 секунд.
NMF:
Найкращі параметри: {'n_epochs': 20, 'n_factors': 20, 'reg_pu': 0.06, 'reg_qi': 0.06}
Найкраща RMSE: 1.1047
Найкраща MAE: 0.8408
Середні часи навчання (Fit time) та часи тестування (Test time) на кожному спліті заняли відповідно приблизно 1.35 та 0.23 секунди.
SVD показує найкращі результати серед усіх трьох алгоритмів з найнижчими значеннями RMSE (0.9517) та MAE (0.7498).

SVD++ показує значно гірші результати порівняно з SVD, з набагато вищими значеннями RMSE (1.7480) та MAE (1.4018). SVD++ враховує неявний відгук, крім явних оцінок, але в даному випадку це не призвело до поліпшення результатів. SVDpp є найповільнішим алгоритмом, так як виконується тривало через високий обчислювальний ресурс, який потрібен для обробки більш складних моделей.

NMF показує кращі результати, ніж SVD++, але гірші, ніж SVD. Його RMSE (1.1047) та MAE (0.8408) нижчі, ніж у SVD++, але все ж вищі, ніж у SVD. NMF має інший підхід до факторизації матриць, який спрямований на розкладання матриць на невід'ємні компоненти.

Загалом, SVD виділяється як найефективніший алгоритм для даного набору даних, надаючи найкращу точність прогнозування для рейтингів фільмів.